In [1]:
import pandas as pd 
import tensorflow as tf

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import time
import string

import chart_studio.plotly
import chart_studio.plotly as py
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
lines = pd.read_excel('eng_hin.xlsx')
lines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2979 entries, 0 to 2978
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Column1  2979 non-null   object
 1   Column2  2979 non-null   object
dtypes: object(2)
memory usage: 46.7+ KB


In [3]:
lines.head()

,Column1,Column2
0,Wow!,वाह!
1,Duck!,झुको!
2,Duck!,बतख़!
3,Help!,बचाओ!
4,Jump.,उछलो.


### preprocessing

In [4]:
def preprocess_eng_sentence(sent):
    '''Function to preprocess English sentence'''
    sent = sent.lower()  # Convert to lowercase
    sent = re.sub("'", '', sent)  # Remove single quotation marks if any
    sent = ' '.join(sent.split())  # Remove extra spaces
    sent = '<start> ' + sent + ' <end>'  # Add '<start>' and '<end>' tokens
    return sent

In [5]:
def preprocess_target_sentence(sent):
    '''Function to preprocess Hindi sentence'''
    sent = sent.lower()  # Convert to lowercase
    sent = re.sub("'", '', sent)  # Remove single quotation marks if any
    sent = ' '.join(sent.split())  # Remove extra spaces
    sent = '<start> ' + sent + ' <end>'  # Add '<start>' and '<end>' tokens
    return sent

In [6]:
# Preprocess sentence pairs
sent_pairs = []
for index, row in lines.iterrows():
    sent_pair = []
    eng = preprocess_eng_sentence(row['Column1'])
    sent_pair.append(eng)
    hin = preprocess_target_sentence(row['Column2'])
    sent_pair.append(hin)
    sent_pairs.append(sent_pair)

print(sent_pairs[:10])

[['<start> wow! <end>', '<start> वाह! <end>'], ['<start> duck! <end>', '<start> झुको! <end>'], ['<start> duck! <end>', '<start> बतख़! <end>'], ['<start> help! <end>', '<start> बचाओ! <end>'], ['<start> jump. <end>', '<start> उछलो. <end>'], ['<start> jump. <end>', '<start> कूदो. <end>'], ['<start> jump. <end>', '<start> छलांग. <end>'], ['<start> hello! <end>', '<start> नमस्ते। <end>'], ['<start> hello! <end>', '<start> नमस्कार। <end>'], ['<start> cheers! <end>', '<start> वाह-वाह! <end>']]


### Create a class to map every word to an index and vice-versa for any given vocabulary.

In [7]:
# This class creates a word -> index mapping (e.g,. "dad" -> 5) and vice-versa 
# (e.g., 5 -> "dad") for each language,
class LanguageIndex():
    def __init__(self, lang):
        self.lang = lang
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()

        self.create_index()

    def create_index(self):
        for phrase in self.lang:
            self.vocab.update(phrase.split(' '))

        self.vocab = sorted(self.vocab)

        self.word2idx['<pad>'] = 0
        for index, word in enumerate(self.vocab):
            self.word2idx[word] = index + 1

        for word, index in self.word2idx.items():
            self.idx2word[index] = word

In [8]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [9]:
def load_dataset(pairs, num_examples):
    # pairs => already created cleaned input, output pairs

    # index language using the class defined above    
    inp_lang = LanguageIndex(en for en, ma in pairs)
    targ_lang = LanguageIndex(ma for en, ma in pairs)
    
    # Vectorize the input and target languages
    
    # English sentences
    input_tensor = [[inp_lang.word2idx[s] for s in en.split(' ')] for en, ma in pairs]
    
    # Hindi sentences
    target_tensor = [[targ_lang.word2idx[s] for s in ma.split(' ')] for en, ma in pairs]
    
    # Calculate max_length of input and output tensor
    # Here, we'll set those to the longest sentence in the dataset
    max_length_inp, max_length_tar = max_length(input_tensor), max_length(target_tensor)
    
    # Padding the input and output tensor to the maximum length
    input_tensor = tf.keras.preprocessing.sequence.pad_sequences(input_tensor, 
                                                                 maxlen=max_length_inp,
                                                                 padding='post')
    
    target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, 
                                                                  maxlen=max_length_tar, 
                                                                  padding='post')
    
    return input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_tar

In [10]:
input_tensor, target_tensor, inp_lang, targ_lang, max_length_inp, max_length_targ = load_dataset(sent_pairs, len(lines))

In [11]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.25, random_state = 101)

# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val)

(2234, 2234, 745, 745)

In [27]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 2000
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 300
units = 128
vocab_inp_size = len(inp_lang.word2idx)
vocab_tar_size = len(targ_lang.word2idx)

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [28]:
def gru(units):

    return tf.keras.layers.GRU(units, 
                                   return_sequences=True, 
                                   return_state=True, 
                                   recurrent_activation='sigmoid', 
                                   recurrent_initializer='glorot_uniform')

### Encoder and Decoder

In [29]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.enc_units)
        
    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)        
        return output, state
    
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [30]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)
        
        # used for attention
        self.W1 = tf.keras.layers.Dense(self.dec_units)
        self.W2 = tf.keras.layers.Dense(self.dec_units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, x, hidden, enc_output):

        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        
        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying tanh(FC(EO) + FC(H)) to self.V
        score = self.V(tf.nn.tanh(self.W1(enc_output) + self.W2(hidden_with_time_axis)))
        
        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)
        
        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * enc_output
        context_vector = tf.reduce_sum(context_vector, axis=1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        
        # passing the concatenated vector to the GRU
        output, state = self.gru(x)
        
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * 1, vocab)
        x = self.fc(output)
        
        return x, state, attention_weights
        
    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.dec_units))

In [31]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

### Optimizer and loss function

In [32]:
custom_learning_rate = 0.05
optimizer = tf.optimizers.Adam(learning_rate=custom_learning_rate)

def loss_function(real, pred):
    mask = 1 - np.equal(real, 0)
    loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
    return tf.reduce_mean(loss_)

In [33]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [34]:
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()
    
    hidden = encoder.initialize_hidden_state()
    total_loss = 0
    
    for (batch, (inp, targ)) in enumerate(dataset):
        loss = 0
        
        with tf.GradientTape() as tape:
            enc_output, enc_hidden = encoder(inp, hidden)
            
            dec_hidden = enc_hidden
            
            dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * BATCH_SIZE, 1)       
            
            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
                
                loss += loss_function(targ[:, t], predictions)
                
                # using teacher forcing
                dec_input = tf.expand_dims(targ[:, t], 1)
        
        batch_loss = (loss / int(targ.shape[1]))
        
        total_loss += batch_loss
        
        variables = encoder.variables + decoder.variables
        
        gradients = tape.gradient(loss, variables)
        
        optimizer.apply_gradients(zip(gradients, variables))
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every epoch
    checkpoint.save(file_prefix = checkpoint_prefix)
    
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss / N_BATCH))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 2.3434
Epoch 1 Loss 2.3434
Time taken for 1 epoch 32.593273639678955 sec

Epoch 2 Batch 0 Loss 2.3209
Epoch 2 Loss 2.3209
Time taken for 1 epoch 40.71084904670715 sec

Epoch 3 Batch 0 Loss 1.9161
Epoch 3 Loss 1.9161
Time taken for 1 epoch 30.06349277496338 sec

Epoch 4 Batch 0 Loss 1.6717
Epoch 4 Loss 1.6717
Time taken for 1 epoch 29.841777563095093 sec

Epoch 5 Batch 0 Loss 1.6253
Epoch 5 Loss 1.6253
Time taken for 1 epoch 31.504647254943848 sec

Epoch 6 Batch 0 Loss 1.9112
Epoch 6 Loss 1.9112
Time taken for 1 epoch 25.738145351409912 sec

Epoch 7 Batch 0 Loss 1.5618
Epoch 7 Loss 1.5618
Time taken for 1 epoch 22.82331109046936 sec

Epoch 8 Batch 0 Loss 1.6117
Epoch 8 Loss 1.6117
Time taken for 1 epoch 23.134917736053467 sec

Epoch 9 Batch 0 Loss 1.5139
Epoch 9 Loss 1.5139
Time taken for 1 epoch 42.27096152305603 sec

Epoch 10 Batch 0 Loss 1.5010
Epoch 10 Loss 1.5010
Time taken for 1 epoch 27.506473064422607 sec



In [35]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

### inference and setup testing

In [36]:
def evaluate(inputs, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    
    attention_plot = np.zeros((max_length_targ, max_length_inp))
    sentence = ''
    for i in inputs[0]:
        if i == 0:
            break
        sentence = sentence + inp_lang.idx2word[i] + ' '
    sentence = sentence[:-1]
    
    inputs = tf.convert_to_tensor(inputs)
    
    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input, dec_hidden, enc_out)
        
        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.idx2word[predicted_id] + ' '

        if targ_lang.idx2word[predicted_id] == '<end>':
            return result, sentence, attention_plot
        
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot


In [37]:
def predict_random_val_sentence():
    actual_sent = ''
    k = np.random.randint(len(input_tensor_val))
    random_input = input_tensor_val[k]
    random_output = target_tensor_val[k]
    random_input = np.expand_dims(random_input,0)
    result, sentence, attention_plot = evaluate(random_input, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)
    print('Input: {}'.format(sentence[8:-6]))
    print('Predicted translation: {}'.format(result[:-6]))
    for i in random_output:
        if i == 0:
            break
        actual_sent = actual_sent + targ_lang.idx2word[i] + ' '
    actual_sent = actual_sent[8:-7]
    print('Actual translation: {}'.format(actual_sent))
    attention_plot = attention_plot[:len(result.split(' '))-2, 1:len(sentence.split(' '))-1]
    sentence, result = sentence.split(' '), result.split(' ')
    sentence = sentence[1:-1]
    result = result[:-2]

    # use plotly to generate the heat map
    trace = go.Heatmap(z = attention_plot, x = sentence, y = result, colorscale='greens')
    data=[trace]
    iplot(data)

In [38]:
predict_random_val_sentence()

Input: i had two cups of coffee.
Predicted translation: वह अपने लिए वह अपने लिए वह अपने लिए वह अपने लिए वह अपने लिए वह अपने लिए वह अपने लिए वह अपने लिए वह अपन
Actual translation: मैंने दो कप कॉफ़ी पी।


In [39]:
predict_random_val_sentence()

Input: she reminds me very much of her mother.
Predicted translation: वह अपने लिए वह अपने लिए वह अपने लिए वह अपने लिए वह अपने लिए वह अपने लिए वह अपने लिए वह अपने लिए वह अपन
Actual translation: वह मुझे उसकी माँ की याद दिलाती है।


In [40]:
predict_random_val_sentence()

Input: i really dont need any help.
Predicted translation: वह अपने लिए वह अपने लिए वह अपने लिए वह अपने लिए वह अपने लिए वह अपने लिए वह अपने लिए वह अपने लिए वह अपन
Actual translation: मुझे सच में किसी की भी मदद की ज़रूरत नहीं है।


In [41]:
predict_random_val_sentence()

Input: nothing but a miracle can save her now.
Predicted translation: वह अपने लिए वह अपने लिए वह अपने लिए वह अपने लिए वह अपने लिए वह अपने लिए वह अपने लिए वह अपने लिए वह अपन
Actual translation: अब तो उसे कोई चमत्कार ही बचा सकता है।
